# LSTM


### Imports

In [1]:
import yaml
import torch
import torch.nn as nn
import torch.nn.functional as F

### Data Loading

In [2]:
cd Downloads

[WinError 2] The system cannot find the file specified: 'Downloads'
C:\Users\ykung\Downloads


In [3]:
cd b4c

C:\Users\ykung\Downloads\b4c


In [4]:
cd Brains4Cars

C:\Users\ykung\Downloads\b4c\Brains4Cars


Straight from github

In [5]:
import os
from os.path import join
import copy

from multiprocessing import Pool
import tqdm

import numpy as np
import pickle
from torch.utils.data import Dataset, DataLoader

ACTION_TO_ID_MAP= {
    'end_action':   0,
    'lchange':      1,
    'lturn':        2,
    'rchange':      3,
    'rturn':        4
}

rng = np.random.default_rng(seed=42)
MAX_FRAMES=150


class B4CDataset(Dataset):
    """
    end_action - 0
    lchange - 1
    lturn - 2
    rchange - 3
    rturn - 4
    """
    def __init__(self, data_cfg, split="train", create_dataset=False):
        self.data_cfg   = data_cfg['DATALOADER_CONFIG']
        self.actions    = self.data_cfg['ACTIONS']
        self.cameras    = self.data_cfg['CAMERAS']
        self.data_dir   = self.data_cfg['DATA_DIR']
        self.split      = split

        videos_dict = self.read_videos_by_action()        

        if create_dataset:
            # self.create_gt_road_labels(videos_dict)
            self.generate_imagesets(videos_dict)
        
        self.image_sets = {}
        splits_process = [self.split] if self.split in ["train", "val", "test"] else ["train", "val", "test"]

        for camera in self.cameras:
            if camera not in self.image_sets:
                self.image_sets[camera] = []

            for curr_split in splits_process:
                imageset_path = join(self.data_dir, f'ImageSets_{camera}', f'{curr_split}.txt')
                self.image_sets[camera].extend([line.strip() for line in open(imageset_path, 'r')])

        # Drop nondivisible videos for fold splits from end
        if "fold" in self.split:
            num_folds = int(self.split.split("_")[0])
            num_to_drop = len(self) % num_folds
            self.image_sets = {k: v[:-num_to_drop] for k, v in self.image_sets.items()}

        print(f'Added {len(self)} videos to the dataset.')

    def read_videos_by_action(self):
        videos_dict = {}
        # Combine image sets for all cameras
        for camera in self.cameras:
            camera_dir = join(self.data_dir, camera+"_processed")

            for action in self.actions:
                action_dir = join(camera_dir, action)

                if action not in videos_dict:
                    videos_dict[action] = []

                # Take the set intersection between all the cameras sequentially
                video_action_set = set([f for f in os.listdir(action_dir) if os.path.isdir(join(action_dir, f))])

                if len(videos_dict[action])==0:
                    videos_dict[action] = video_action_set
                else:
                    videos_dict[action] = videos_dict[action].intersection(video_action_set)

        # Convert videos_dict back to list
        for action in self.actions:
            videos_dict[action] = list(videos_dict[action])

        # Check that files exist
        for camera in self.cameras:
            camera_dir = join(self.data_dir, camera+"_processed")

            for action in self.actions:
                action_dir = join(camera_dir, action)

                for video_dir in videos_dict[action]:
                    video_dir = join(action_dir, video_dir)
                    assert os.path.exists(video_dir), f"Video directory {video_dir} does not exist."

        print("Finished reading all valid videos by directory.")
        return videos_dict

    @staticmethod
    def write_list_to_file(file_path, data_list):
        """
        Create/overwrite a .txt file and write each line of the Python list to a new line in the file.

        Parameters:
            file_path : str
                The path to the .txt file.
            data_list : list
                The Python list containing data to write to the file.
        """
        with open(file_path, 'w') as file:
            file.writelines(f"{item}\n" for item in data_list)

    def check_data_quality(self, video_subdirs, camera):
        """
        Check that all videos have the same number of frames and that the number of frames is less than MAX_FRAMES.

        Parameters:
            videos_dict : dict
                Dictionary containing the list of videos for each action.
        """

        valid_videos_mask = np.zeros(len(video_subdirs), dtype=bool)
        for video_idx, video_subdir in enumerate(video_subdirs):
            video_path = join(self.data_dir, video_subdir)
            data_dict = {}

            # Check that all videos have the full frame set
            if camera=="face":
                data_dict['gt_gazepose'] = self.get_face_labels(video_path)
                valid_videos_mask[video_idx] = len(data_dict['gt_gazepose'])>=MAX_FRAMES-1 # gazepose has 149
            elif camera=="road":
                data_dict['gt_bbox'], data_dict['gt_lanes'] = self.get_road_labels(video_path)
                valid_videos_mask[video_idx] = len(data_dict['gt_bbox'])>=MAX_FRAMES and len(data_dict['gt_lanes'])>=MAX_FRAMES

            if valid_videos_mask[video_idx]==0:
                print(f'Video {video_subdir} does not have the full frame set, skipping...')

        return valid_videos_mask
            

    def generate_imagesets(self, videos_dict):    
        print("Generating imagesets...")
        facecam_imageset_dict = {'train': [], 'val': [], 'test': []}
        roadcam_imageset_dict = copy.deepcopy(facecam_imageset_dict)
        train_pct, val_pct, test_pct = 0.7, 0.15, 0.15
        for action, action_videos in videos_dict.items():
            road_cam_action_dir = join('road_camera_processed_combined', action)
            road_cam_video_labels = np.array([join(road_cam_action_dir, f) for f in action_videos])
            road_cam_video_mask = self.check_data_quality(road_cam_video_labels, "road")

            face_cam_action_dir = join('face_camera_processed', action)
            face_cam_video_labels = np.array([join(face_cam_action_dir, f) for f in action_videos])
            face_cam_video_mask = self.check_data_quality(face_cam_video_labels, "face")

            combined_cam_video_mask = np.logical_and(road_cam_video_mask, face_cam_video_mask)
            road_cam_video_labels = road_cam_video_labels[combined_cam_video_mask]
            face_cam_video_labels = face_cam_video_labels[combined_cam_video_mask]

            road_cam_video_labels_sort_idx = np.argsort(road_cam_video_labels)
            road_cam_video_labels = road_cam_video_labels[road_cam_video_labels_sort_idx]
            face_cam_video_labels = face_cam_video_labels[road_cam_video_labels_sort_idx]

            # Ensure file order matches for road and face camera
            for i in range(len(road_cam_video_labels)):
                assert os.path.basename(road_cam_video_labels[i])==os.path.basename(face_cam_video_labels[i]), 'Video labels do not match'
            num_videos = len(road_cam_video_labels)

            num_train       = int(num_videos * train_pct)
            num_val         = int(num_videos * val_pct)
            num_test        = int(num_videos * test_pct)

            indices = np.arange(0, num_videos, 1)
            rng.shuffle(indices)

            videos_indices = {"train": [], "val": [], "test": []}
            videos_indices['train'], videos_indices['val'], videos_indices['test'] = indices[:num_train], \
                indices[num_train:num_train+num_val],  indices[num_train+num_val:num_train+num_val+num_test]

            for split in facecam_imageset_dict.keys():
                roadcam_imageset_dict[split].extend(road_cam_video_labels[videos_indices[split]].tolist())
                facecam_imageset_dict[split].extend(face_cam_video_labels[videos_indices[split]].tolist())

        # Dump to imageset files for road and face camera
        roadcam_imagesets_dir = join(self.data_dir, "ImageSets_road_camera")
        facecam_imagesets_dir = join(self.data_dir, "ImageSets_face_camera")
        if not os.path.exists(roadcam_imagesets_dir):
            print(f'Video root directory {roadcam_imagesets_dir} does not exist. Creating...')
            os.mkdir(roadcam_imagesets_dir)
        if not os.path.exists(facecam_imagesets_dir):
            print(f'Video root directory {facecam_imagesets_dir} does not exist. Creating...')
            os.mkdir(facecam_imagesets_dir)

        for split_key in facecam_imageset_dict.keys():
            road_cam_split_path = join(roadcam_imagesets_dir, f'{split_key}.txt')
            face_cam_split_path = join(facecam_imagesets_dir, f'{split_key}.txt')
            print(f'Saving imageset file {split_key} for road {road_cam_split_path} and {face_cam_split_path}')
            self.write_list_to_file(road_cam_split_path, roadcam_imageset_dict[split_key])
            self.write_list_to_file(face_cam_split_path, facecam_imageset_dict[split_key])

    def __len__(self):
        num_files = 0
        for camera in self.cameras: 
            assert num_files==0 or num_files==len(self.image_sets[camera]), "Number files in dataset not correct"
            num_files=len(self.image_sets[camera])
        return num_files
    
    def collate_fn(self, data):
        # print(data[0][1])
        data_batch = [bi[0] for bi in data]
        action_batch = [bi[1] for bi in data]
        return data_batch, action_batch

    def get_face_labels(self, label_dir):  
        gazepose_path = join(label_dir, 'gazepose.npy')
        assert os.path.exists(gazepose_path), f'Label file {gazepose_path} does not exist'
        gt_gazepose = np.load(gazepose_path)

        if gt_gazepose.shape[0] < MAX_FRAMES:
            # print(f'Gaze pose {gazepose_path} has less than 150 frames, padding with zeros...')
            # Pad with zeros
            gt_gazepose = np.pad(gt_gazepose, ((0, MAX_FRAMES-gt_gazepose.shape[0]), (0, 0)), mode='constant')
        # Assume gt_gazepose is not smaller than MAX_FRAMES
        num_frames = min(MAX_FRAMES, gt_gazepose.shape[0])
        gt_gazepose = gt_gazepose[:num_frames, :]

        return gt_gazepose

    def get_road_labels(self, label_dir):
        
        bbox_file = join(label_dir, 'bbox_labels.pkl')
        lane_file = join(label_dir, 'lane_labels.pkl')

        assert os.path.exists(bbox_file), f'Label directory {bbox_file} does not exist'
        assert os.path.exists(lane_file), f'Label directory {lane_file} does not exist'

        # Load bbox detections
        with open(bbox_file, 'rb') as f:
            gt_bbox = pickle.load(f)
        # Load road labels
        with open(lane_file, 'rb') as f:
            gt_lanes = pickle.load(f)

        gt_bbox = gt_bbox[:MAX_FRAMES]
        gt_lanes = gt_lanes[:MAX_FRAMES]

        return gt_bbox, gt_lanes
    
    def get_action_label(self, video_dir):
        action_label = video_dir.split('/')[-2]
        assert action_label in ACTION_TO_ID_MAP.keys(), f'Action {action_label} not in action map'
        action_id = ACTION_TO_ID_MAP[action_label]
        return action_id

    @staticmethod
    def combine_img_labels(args):
        split_video_path, combined_video_path = args
        img_label_files = [f for f in os.listdir(split_video_path) if f.endswith('.pkl')]

        MAX_NUM_BBOXES = 5 
        full_img_label_np = np.ones((MAX_FRAMES+1, MAX_NUM_BBOXES*5)) * -1
        num_img_label_files = len(img_label_files)
        for img_label_idx, img_label_file in enumerate(img_label_files):
            img_label_path = join(split_video_path, img_label_file)
            with open(img_label_path, 'rb') as f:
                img_data = pickle.load(f)
                IMG_W, IMG_H = 720, 480

                #1 Convert to xc, yc, w, h
                x1, x2, y1, y2 = img_data['xyxy'][:, 0], img_data['xyxy'][:, 2], img_data['xyxy'][:, 1], img_data['xyxy'][:, 3]
                xc = (x1 + x2) / 2
                yc = (y1 + y2) / 2
                h = (x2 - x1)
                w = (y2 - y1)
            
                #2 Only keep bboxes with h and w < 0.33 (Ignore large bboxes of self)
                bbox_size_mask = np.logical_and(h < IMG_W*0.33, w < IMG_H*0.33)

                #3 Only keep labels with class_ids = 0, 1, 2, 3, 4 (Ignore 5 Date)
                class_ids = img_data['class_id'] 
                class_ids_mask = np.logical_and(class_ids >= 0, class_ids <= 4)

                bbox_mask = np.logical_and(bbox_size_mask, class_ids_mask)
                proc_img_label = np.ones((MAX_NUM_BBOXES, 5), dtype=int)*-1 # max of five bbox detections per image

                if np.sum(bbox_mask)>0:
                    xc = xc[bbox_mask].astype(int)
                    yc = yc[bbox_mask].astype(int)
                    h = h[bbox_mask].astype(int)
                    w = w[bbox_mask].astype(int)
                    class_ids = class_ids[bbox_mask]
                    gt_objs = np.stack((xc, yc, w, h, class_ids), axis=1)

                    #4 Select top 5 largest boxes
                    gt_obj_areas = gt_objs[:, 2] * gt_objs[:, 3]
                    gt_objs_sort_idx = np.argsort(-gt_obj_areas, kind='stable') # Sort high to low
                    num_objs = min(MAX_NUM_BBOXES, len(gt_objs_sort_idx))

                    proc_img_label[:num_objs, :] = gt_objs[gt_objs_sort_idx[:num_objs], :]

                proc_img_label = proc_img_label.flatten()
                proc_img_label = np.expand_dims(proc_img_label, axis=0)
                full_img_label_np[img_label_idx] = proc_img_label

        assert os.path.exists(combined_video_path), f'Video label directory {combined_video_path} does not exist'
        video_path = join(combined_video_path, 'bbox_labels.pkl')
        with open(video_path, 'wb') as f:
            pickle.dump(full_img_label_np, f)
        print("Saved combined bbox labels to: ", video_path)

        # Load, pad, save lane dets
        original_label_dir = split_video_path.replace('road_camera_processed', 'road_camera').replace('labels/', '')
        road_path = join(original_label_dir+".txt")
        gt_lanes = np.loadtxt(road_path, delimiter=',', dtype=int).reshape(1, -1)
        gt_lanes_padded = np.ones((MAX_FRAMES, 3)) * -1
        num_valid_gt_lanes = min(num_img_label_files, MAX_FRAMES)
        gt_lanes_padded[:num_valid_gt_lanes] = np.repeat(gt_lanes, [num_valid_gt_lanes], axis=0)

        lanes_path = join(combined_video_path, 'lane_labels.pkl')
        with open(lanes_path, 'wb') as f:
            pickle.dump(gt_lanes_padded, f)

    def create_gt_road_labels(self, videos_dict):
        split_video_path_list = []
        combined_video_path_list = []
        for action, action_videos in videos_dict.items():
            for video in action_videos:
                video_path = join(self.data_dir, 'road_camera_processed', 'labels', action, video)
                if not os.path.exists(video_path):
                    print(f'Video path {video_path} does not exist')
                    continue
                assert os.path.exists(video_path), f'Video path {video_path} does not exist'

                video_label_dir = join(self.data_dir, "road_camera_processed_combined", action, video)
                if not os.path.exists(video_label_dir):
                    print("Creating directory: ", video_label_dir)
                    os.makedirs(video_label_dir)
                
                split_video_path_list.append(video_path)
                combined_video_path_list.append(video_label_dir)
            # self.combine_img_labels((split_video_path_list[0], combined_video_path_list[0]))
        pool = Pool(processes=16)
        for _ in tqdm.tqdm(pool.imap_unordered(self.combine_img_labels, zip(split_video_path_list, 
            combined_video_path_list)), total=len(split_video_path_list)):
            pass

    def __getitem__(self, idx):
        data_dict = {}
        action_id=None

        for camera in self.cameras:
            video_subdir    = self.image_sets[camera][idx]
            video_fulldir   = join(self.data_dir, video_subdir)

            if camera == 'face_camera':
                data_dict['gt_gazepose'] = self.get_face_labels(video_fulldir)
            elif camera == 'road_camera':
                # Load all pickle files in the video directory
                data_dict['gt_bbox'], data_dict['gt_lanes'] = self.get_road_labels(video_fulldir)

            if action_id is None:
                action_id = self.get_action_label(video_fulldir)

        # Sort dict by key so that it is is consistent
        sorted_data_dict = dict(sorted(data_dict.items(), key=lambda item: item[0]))

        sorted_gt_np = np.empty((MAX_FRAMES, 0))
        # Stack all values from data_dict into a single np array

        for _, items in sorted_data_dict.items():
            sorted_gt_np = np.hstack((sorted_gt_np, items))

        # TOOD: Extract action label from Imageset file
        #  150 x [ (5x5) (2) (2) (3) ] # Pad if not enough frames obj_detections, gazepose, lane_detections # 150 x 32
        return sorted_gt_np, action_id # processed_input, action_label one hot vector 

config file

In [6]:
cfg=None
with open("C:/Users/ykung/Downloads/lstm_all.yaml", 'r') as file:
    cfg = yaml.safe_load(file)

In [7]:
batch = 50

make dataset (do once)

In [8]:
dataset = B4CDataset(cfg, split="5_fold", create_dataset=False)

Finished reading all valid videos by directory.
Added 585 videos to the dataset.


In [10]:
#dataloader = DataLoader(dataset, collate_fn=dataset.collate_fn, batch_size=batch, shuffle=True)

In [10]:
#valdataset = B4CDataset(cfg, split="val", create_dataset=False)
#val_loader = DataLoader(valdataset, collate_fn=dataset.collate_fn, batch_size=1, shuffle=True)

Finished reading all valid videos by directory.
Added 87 videos to the dataset.


In [11]:
#testdataset = B4CDataset(cfg, split="test", create_dataset=False)
#test_loader = DataLoader(testdataset, collate_fn=dataset.collate_fn, batch_size=1, shuffle=True)

Finished reading all valid videos by directory.
Added 87 videos to the dataset.


#### Define your network, loss function and optimizer

In [9]:
class Net(nn.Module):
    def __init__(self, emb_dim=10, hidden_dim=100):
        super().__init__()
        self.fc1 = nn.Linear(1500,100)
        self.fc2 = nn.Linear(100,5)
        self.lstmgaze = nn.LSTM(4,10,batch_first=True)
        #self.lstmlane = nn.LSTM(3,5,batch_first=True)
        #self.lstmobj = nn.LSTM(25,10,batch_first=True)
        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()
        self.double()

    def forward(self, b):
        #b1, (h1, c1) = self.lstmobj(b[:,:,0:25])
        b2, (h2, c2) = self.lstmgaze(b[:,:,25:29])
        #b3, (h3, c3) = self.lstmlane(b[:,:,29:32])
        #b1 = torch.flatten(b1, start_dim=1)
        b2 = torch.flatten(b2, start_dim=1)
        #b3 = torch.flatten(b3, start_dim=1)
        #b = torch.cat((b1,b2,b3), 1)
        b = self.fc1(b2)
        b = self.relu(b)
        b = self.fc2(b)
        b = self.sig(b)
        return b
        

In [10]:
net1 = Net()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net1.parameters(), lr=0.01, momentum=0.9)

#### Implement the training loop function

In [11]:
import random
# k-split validation
totalsize = len(dataset)
indices = list(range(0,totalsize))
random.shuffle(indices)
seg = int(totalsize/10) # number of splits



In [12]:
split = 1 #change for each k-validation split

trainlefti = indices[0:split*seg]
trainrighti = indices[min(split*seg + seg,totalsize):totalsize]
traini = trainlefti + trainrighti
vali = indices[split*seg:min(totalsize,seg*split+seg)]

train_set = torch.utils.data.dataset.Subset(dataset,traini)
val_set = torch.utils.data.dataset.Subset(dataset,vali)

trainloader = DataLoader(train_set, collate_fn=dataset.collate_fn, batch_size=batch, shuffle=True)
valloader = DataLoader(val_set, collate_fn=dataset.collate_fn, batch_size=1, shuffle=True)

In [13]:
def train(net, trainloader, optimizer, loss_fn, num_epochs):
    # Put the network in training mode
    net.train()

    # Training loop
    for epoch in range(num_epochs):
        running_loss = 0
        for batch_idx, (data, targets) in enumerate(trainloader):
            # TODO: zero the parameter gradients + forward pass + loss computation + backward pass + weight update
            data = np.array(data)
            tdata = torch.tensor(data)
            optimizer.zero_grad()
            prediction = net(tdata)
            ttargets = torch.tensor(targets)
            loss = loss_fn(prediction, ttargets)           
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(running_loss)

Evaluate

In [14]:
import sklearn
from sklearn import metrics

In [15]:
def eval(net, loader):
    net.eval()
    se = 0 # sum of correct pred
    total = 0
    f1input = np.zeros(100)
    f1target = np.zeros(100)
    i1 = 0
    mt = np.zeros(5)
    mr = np.zeros(5)
    for batch_idx, (data, targets) in enumerate(loader):
        data = np.array(data)
        tdata = torch.tensor(data)
        y = net(tdata)
        f1input[i1] = int(torch.argmax(y))
        f1target[i1] = int(targets[0])
        i1 += 1
        if int(torch.argmax(y)) == targets[0]:
            se += 1
            mr[int(torch.argmax(y))] += 1 
        total += 1
        mt[int(torch.argmax(y))] += 1
        
    f1score = sklearn.metrics.f1_score(f1target[0:i1], f1input[0:i1], average = 'macro')
    return float(se/total), f1score, mt, mr

In [16]:
evalmax = np.zeros((2,10))

In [18]:
totalcorrect = np.zeros(5)
totalneeded = np.zeros(5)
for split in range(10):
    print(split)
    permval = np.zeros(5)
    permval2 = np.zeros(5)
    net1 = Net()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net1.parameters(), lr=0.01, momentum=0.9)
    
    trainlefti = indices[0:split*seg]
    trainrighti = indices[min(split*seg + seg,totalsize):totalsize]
    traini = trainlefti + trainrighti
    vali = indices[split*seg:min(totalsize,seg*split+seg)]

    train_set = torch.utils.data.dataset.Subset(dataset,traini)
    val_set = torch.utils.data.dataset.Subset(dataset,vali)

    trainloader = DataLoader(train_set, collate_fn=dataset.collate_fn, batch_size=batch, shuffle=True)
    valloader = DataLoader(val_set, collate_fn=dataset.collate_fn, batch_size=1, shuffle=True)
    
    for i in range(100): # for train/val stationary time before manuever
        train(net1,trainloader,optimizer,loss_fn,1)
        evalval, f1val, newval1, newval2 = eval(net1,valloader)
        print('eval: '+str(evalval) + '         f1score: ' + str(f1val))
        if evalval > evalmax[0,split]:
            evalmax[0,split] = evalval
            evalmax[1,split] = f1val
            permval = newval1
            permval2 = newval2
            
    totalcorrect = np.add(totalcorrect,permval2)
    totalneeded = np.add(totalneeded,permval)
    
    

0
17.433837368621813
eval: 0.3103448275862069         f1score: 0.096
16.45555674990859
eval: 0.3448275862068966         f1score: 0.10389610389610389
16.1020940947409
eval: 0.3620689655172414         f1score: 0.1538961038961039
15.571852133774385
eval: 0.3448275862068966         f1score: 0.15000000000000002
15.117839648657876
eval: 0.3620689655172414         f1score: 0.12666666666666665
14.685792815673036
eval: 0.3620689655172414         f1score: 0.14079794079794078
14.857385082826857
eval: 0.5344827586206896         f1score: 0.33312169312169315
14.593304502162809
eval: 0.5344827586206896         f1score: 0.336319612590799
14.404014462769105
eval: 0.5         f1score: 0.30060606060606054
14.16263948220565
eval: 0.5862068965517241         f1score: 0.3789463376419898
14.02218401603449
eval: 0.603448275862069         f1score: 0.37593984962406013
13.8156807002528
eval: 0.5862068965517241         f1score: 0.36602400309717387
13.960066566259847
eval: 0.6206896551724138         f1score: 0.3853

15.97856693651381
eval: 0.4827586206896552         f1score: 0.13023255813953488
15.78925312889295
eval: 0.4827586206896552         f1score: 0.16345381526104422
15.45328330124536
eval: 0.5172413793103449         f1score: 0.22153846153846152
15.30113388437857
eval: 0.5         f1score: 0.21333333333333332
15.204655067070902
eval: 0.5689655172413793         f1score: 0.2798830409356725
15.134618296665217
eval: 0.603448275862069         f1score: 0.32627337885406504
14.91179111277686
eval: 0.5689655172413793         f1score: 0.29314553990610326
14.835013871303499
eval: 0.5344827586206896         f1score: 0.24530244530244533
14.641771162414797
eval: 0.603448275862069         f1score: 0.3252120912610164
14.466429456614833
eval: 0.603448275862069         f1score: 0.33804511278195487
14.336689556340401
eval: 0.5172413793103449         f1score: 0.3150151240169389
14.50421681494533
eval: 0.5689655172413793         f1score: 0.2843045843045843
14.288755297822473
eval: 0.6379310344827587         f1sc

15.416425179343765
eval: 0.43103448275862066         f1score: 0.21767605633802817
15.274846355916422
eval: 0.43103448275862066         f1score: 0.21767605633802817
14.95256815135499
eval: 0.5         f1score: 0.29885160662851157
14.831812471949974
eval: 0.5517241379310345         f1score: 0.3467391304347826
14.684710507733211
eval: 0.5862068965517241         f1score: 0.37855268975681655
14.529249122674063
eval: 0.6206896551724138         f1score: 0.49755070303894255
14.400207268523099
eval: 0.6206896551724138         f1score: 0.49353400222965443
14.347314647900223
eval: 0.6379310344827587         f1score: 0.5106666666666667
14.142490199625605
eval: 0.6551724137931034         f1score: 0.5701818181818182
13.99539577995289
eval: 0.6551724137931034         f1score: 0.5148571428571429
13.892450908326099
eval: 0.6379310344827587         f1score: 0.5386666666666666
13.911983735443481
eval: 0.6379310344827587         f1score: 0.5048484848484849
13.692009359337558
eval: 0.6379310344827587      

14.417160227708505
eval: 0.5517241379310345         f1score: 0.2692743180648482
14.3148506646496
eval: 0.5344827586206896         f1score: 0.25798319327731095
14.128971791654898
eval: 0.5344827586206896         f1score: 0.25798319327731095
14.068230405294004
eval: 0.5689655172413793         f1score: 0.33793103448275863
14.041135810548028
eval: 0.5689655172413793         f1score: 0.33793103448275863
13.887365838742852
eval: 0.5517241379310345         f1score: 0.32658137882018484
13.895157185193936
eval: 0.5689655172413793         f1score: 0.33538461538461545
13.763136529333359
eval: 0.5689655172413793         f1score: 0.33793103448275863
13.724980653009524
eval: 0.5689655172413793         f1score: 0.33538461538461545
13.634551484828064
eval: 0.5689655172413793         f1score: 0.33793103448275863
13.681274648900612
eval: 0.5689655172413793         f1score: 0.33793103448275863
13.644962338265469
eval: 0.5862068965517241         f1score: 0.3876013641530883
13.510627505139468
eval: 0.56896

14.853475431423057
eval: 0.3793103448275862         f1score: 0.16969696969696968
14.882199520691842
eval: 0.41379310344827586         f1score: 0.21414141414141413
14.931738732838422
eval: 0.3793103448275862         f1score: 0.18753204893414402
14.724417907683021
eval: 0.43103448275862066         f1score: 0.22849614153961978
14.792273662079712
eval: 0.41379310344827586         f1score: 0.21064935064935067
14.698224292654071
eval: 0.39655172413793105         f1score: 0.2019703852280102
14.695508533595646
eval: 0.39655172413793105         f1score: 0.20993589743589744
14.524861530607224
eval: 0.3793103448275862         f1score: 0.20886446886446888
14.464928877792358
eval: 0.39655172413793105         f1score: 0.21631871467937044
14.512309709099085
eval: 0.3793103448275862         f1score: 0.20886446886446888
14.41494765932586
eval: 0.43103448275862066         f1score: 0.25811138014527846
14.365651425502428
eval: 0.41379310344827586         f1score: 0.22470046082949308
14.30180452837686
eval

13.777693981959606
eval: 0.7068965517241379         f1score: 0.43392857142857144
13.872064279579487
eval: 0.7241379310344828         f1score: 0.44453781512605034
13.678495917701573
eval: 0.7586206896551724         f1score: 0.47230731225296446
13.4435745090445
eval: 0.7068965517241379         f1score: 0.43080745341614907
13.319370184765067
eval: 0.7241379310344828         f1score: 0.4441558441558442
13.351383055373384
eval: 0.7586206896551724         f1score: 0.467741935483871
13.27167428321768
eval: 0.7241379310344828         f1score: 0.43775644645209866
13.174513653976476
eval: 0.7413793103448276         f1score: 0.5166666666666667
13.819905532507857
eval: 0.7241379310344828         f1score: 0.42071846282372594
13.760989024533734
eval: 0.7241379310344828         f1score: 0.5556521739130436
13.459403037052514
eval: 0.7241379310344828         f1score: 0.566278677462888
13.194174144356051
eval: 0.7413793103448276         f1score: 0.5818412698412698
13.04635235015844
eval: 0.7241379310344

14.311486076957989
eval: 0.5172413793103449         f1score: 0.3223809523809524
14.296840533763975
eval: 0.5172413793103449         f1score: 0.2480952380952381
14.247482347415026
eval: 0.5344827586206896         f1score: 0.2569503980404164
14.214167349637663
eval: 0.5344827586206896         f1score: 0.333167701863354
14.23025617377123
eval: 0.5172413793103449         f1score: 0.251316595223515
14.09011036903801
eval: 0.5517241379310345         f1score: 0.3388819875776397
14.019223981344712
eval: 0.5517241379310345         f1score: 0.342483994878361
13.90698267184705
eval: 0.5344827586206896         f1score: 0.333167701863354
13.83181782515684
eval: 0.5689655172413793         f1score: 0.3777489177489177
13.725926539217866
eval: 0.5344827586206896         f1score: 0.333167701863354
13.729879730319896
eval: 0.5862068965517241         f1score: 0.3875117370892019
13.683108887926306
eval: 0.5862068965517241         f1score: 0.3875117370892019
13.600005377099315
eval: 0.603448275862069       

13.313496370109796
eval: 0.4482758620689655         f1score: 0.2819268781532932
13.511008329163865
eval: 0.4482758620689655         f1score: 0.27952254641909813
13.481884040743795
eval: 0.41379310344827586         f1score: 0.2652456239412761
13.465878703663556
eval: 0.41379310344827586         f1score: 0.2999209486166008
13.37010156318698
eval: 0.41379310344827586         f1score: 0.29402401372212694
13.33378745839411
eval: 0.4482758620689655         f1score: 0.325024154589372
13.209836246610651
eval: 0.4482758620689655         f1score: 0.2864782276546982
13.16519321094843
eval: 0.4482758620689655         f1score: 0.29059755836367535
13.082166800968391
eval: 0.4827586206896552         f1score: 0.4144541365680796
13.179573068884414
eval: 0.4827586206896552         f1score: 0.40124579124579124
13.12404553031423
eval: 0.4482758620689655         f1score: 0.3632683982683983
13.071634106463366
eval: 0.46551724137931033         f1score: 0.3936267436267436
13.036558582859355
eval: 0.4482758620

13.18928941021754
eval: 0.5344827586206896         f1score: 0.32227886056971516
13.178939248475396
eval: 0.5172413793103449         f1score: 0.3170484687726067
13.091154673630786
eval: 0.5517241379310345         f1score: 0.340665684107099
13.124080152912711
eval: 0.5344827586206896         f1score: 0.3271949718136461
13.036835165592816
eval: 0.5689655172413793         f1score: 0.3448275862068966
13.02854469161004
eval: 0.5344827586206896         f1score: 0.3293040293040293
13.04643715170908
eval: 0.5517241379310345         f1score: 0.3370175438596491
13.017241281269747
eval: 0.5517241379310345         f1score: 0.3300540407762221
13.113769226042486
eval: 0.5689655172413793         f1score: 0.3474453451240481
13.007485399368008
eval: 0.5689655172413793         f1score: 0.3474453451240481
12.97897952265373
eval: 0.5862068965517241         f1score: 0.3569105164258407
12.930071690125223
eval: 0.5344827586206896         f1score: 0.3246841294298921
12.879071020787773
eval: 0.5689655172413793 

12.818047419500754
eval: 0.6551724137931034         f1score: 0.48711201079622135
12.775445147475512
eval: 0.7068965517241379         f1score: 0.5223552123552124
12.808261314827272
eval: 0.6724137931034483         f1score: 0.49740124740124736
12.769504017791494
eval: 0.6724137931034483         f1score: 0.49740124740124736
12.677935187267883
eval: 0.6724137931034483         f1score: 0.49740124740124736
12.648621209518295
eval: 0.6379310344827587         f1score: 0.4715305832147938
12.732463062811107
eval: 0.6379310344827587         f1score: 0.46545391545391546
12.808145000989034
eval: 0.6206896551724138         f1score: 0.45524608819345663
12.558762394367209
eval: 0.6379310344827587         f1score: 0.4715305832147938
12.614665399932484
eval: 0.6379310344827587         f1score: 0.46479483500717367
12.631628779294001
eval: 0.6206896551724138         f1score: 0.45444444444444443
12.616500084347045
eval: 0.6379310344827587         f1score: 0.46545391545391546
12.503695138056749
eval: 0.6551

In [45]:
for i in range(100): # for train/val stationary time before manuever
    print(i)
    train(net1,trainloader,optimizer,loss_fn,1)
    evalval, f1val = eval(net1,valloader)
    if evalval > evalmax[0,split]:
        evalmax[0,split] = evalval
        evalmax[1,split] = f1val
    print('eval: '+str(evalval) + '         f1score: ' + str(f1val))

0
17.71498657224132
eval: 0.08620689655172414         f1score: 0.057777777777777775
1
17.71641240941441
eval: 0.08620689655172414         f1score: 0.057777777777777775
2
17.714181808371997
eval: 0.08620689655172414         f1score: 0.057777777777777775
3
17.716480368383742
eval: 0.08620689655172414         f1score: 0.057777777777777775
4
17.71560913268089
eval: 0.08620689655172414         f1score: 0.057777777777777775
5
17.713946240726123
eval: 0.08620689655172414         f1score: 0.057777777777777775
6
17.71655483202229
eval: 0.08620689655172414         f1score: 0.057777777777777775
7


KeyboardInterrupt: 

In [25]:
np.average(evalmax[0,:])

0.6620689655172414

In [19]:
totalcorrect[0]/totalneeded[0]

0.6733333333333333

In [20]:
totalcorrect[1]/totalneeded[1]

0.6434108527131783

In [21]:
totalcorrect[2]/totalneeded[2]

0.6923076923076923

In [22]:
totalcorrect[3]/totalneeded[3]

0.6554621848739496

In [23]:
totalcorrect[4]/totalneeded[4]

0.631578947368421

In [149]:
torch.save(net1.state_dict(), "C:/Users/ykung/Downloads/b4c/LSTMFull91")

In [155]:
# the values recorded from the best models from each split
recacc = [0.89655,0.89655,0.8793,0.862,0.8448,0.8793,0.8448,0.8621,0.9138,0.8448]
recf1 = [0.9106,0.84262,0.870255,0.8374,0.8449,0.88,0.874,0.8202,0.8905,0.79139]

In [48]:
print(np.average(evalmax[0,:]))
print(np.std(evalmax[0,:]))
print(np.average(evalmax[1,:]))
print(np.std(evalmax[1,:]))

0.6810344827586206
0.06599192830691253
0.5262910412760419
0.11459412364428125


#### Varying time to manuever

In [16]:
def train2(net, trainloader, optimizer, loss_fn, num_epochs):
    # Put the network in training mode
    net.train()

    # Training loop
    for epoch in range(num_epochs):
        running_loss = 0
        for batch_idx, (data, targets) in enumerate(trainloader):
            # TODO: zero the parameter gradients + forward pass + loss computation + backward pass + weight update
            data = np.array(data)
            tdata = torch.tensor(data)
            ttargets = torch.tensor(targets)
            zten = torch.zeros((len(data),30,32))
            
            optimizer.zero_grad()
            prediction = net(tdata)
            loss = loss_fn(prediction, ttargets)           
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            optimizer.zero_grad()
            prediction = net(torch.cat((zten,tdata[:,0:120,:]), 1))
            loss = loss_fn(prediction, ttargets)           
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
                             
            optimizer.zero_grad()
            prediction = net(torch.cat((zten,zten,tdata[:,0:90,:]), 1))
            loss = loss_fn(prediction, ttargets)           
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            optimizer.zero_grad()
            prediction = net(torch.cat((zten,zten,zten,tdata[:,0:60,:]), 1))
            loss = loss_fn(prediction, ttargets)           
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
                             
            optimizer.zero_grad()
            prediction = net(torch.cat((zten,zten,zten,zten,tdata[:,0:30,:]), 1))
            loss = loss_fn(prediction, ttargets)           
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
                             
        print(running_loss)

In [22]:
for i in range(50): # train/val for varying time to manuever
    print(i)
    train2(net1,trainloader,optimizer,loss_fn,1)
    evalval, f1val = eval(net1,valloader)
    print('eval: '+str(evalval) + '         f1score: ' + str(f1val))
    if evalval > 0.80:
        break

0
84.34658307743275
eval: 0.3448275862068966         f1score: 0.10256410256410257
1
79.30189798016265
eval: 0.3448275862068966         f1score: 0.10256410256410257
2
77.39387576021282
eval: 0.39655172413793105         f1score: 0.21077625570776254
3
74.902875383899
eval: 0.43103448275862066         f1score: 0.25396825396825395
4
73.72999251237188
eval: 0.4482758620689655         f1score: 0.2849348435026184
5
72.08498041827535
eval: 0.5344827586206896         f1score: 0.4403404791929382
6
71.00632982535292
eval: 0.5517241379310345         f1score: 0.4097716718266254
7
70.45337460459773
eval: 0.5         f1score: 0.41238702817650186
8
69.05921841629018
eval: 0.603448275862069         f1score: 0.4826839826839827
9
68.15351578436348
eval: 0.6379310344827587         f1score: 0.5632851359167148
10
67.1173762313591
eval: 0.6896551724137931         f1score: 0.6489137336093858
11
66.36479803751492
eval: 0.7068965517241379         f1score: 0.6834023119737405
12
65.91238094103313
eval: 0.655172413

In [19]:
def eval2(net, loader):
    net.eval()
    t2m = np.zeros(1,100)
    totalcorrect = np.zeros((1,5,100))
    i1 = 0
    for batch_idx, (data, targets) in enumerate(loader):
        data = np.array(data)
        tdata = torch.tensor(data)
        zten = torch.zeros((len(data),30,32))
        
        y = net(torch.cat((zten,zten,zten,zten,tdata[:,0:30,:]), 1))
        if int(torch.argmax(y)) == targets[0]:
            t2m[i1] = 5
            totalcorrect[0,i1] = 1
            
        y = net(torch.cat((zten,zten,zten,tdata[:,0:60,:]), 1))
        if int(torch.argmax(y)) == targets[0]:
            if t2m[i1] == 0:
                t2m[i1] = 4
            totalcorrect[1,i1] = 1
            
        y = net(torch.cat((zten,zten,tdata[:,0:90,:]), 1))
        if int(torch.argmax(y)) == targets[0]:
            if t2m[i1] == 0:
                t2m[i1] = 3
            totalcorrect[2,i1] = 1
            
        y = net(torch.cat((zten,tdata[:,0:120,:]), 1))
        if int(torch.argmax(y)) == targets[0]:
            if t2m[i1] == 0:
                t2m[i1] = 2
            totalcorrect[3,i1] = 1
            
        y = net(tdata)
        if int(torch.argmax(y)) == targets[0]:
            if t2m[i1] == 0:
                t2m[i1] = 1
            totalcorrect[4,i1] = 1
            
        i1+=1
        
    return t2m[0:i1], totalcorrect[:,0:i1]

In [112]:
time_to_maneuver, correct_expansion = eval2(net1, valloader)

In [113]:
lt2m = len(time_to_maneuver)
print('Total maneuvers: ' + str(lt2m))
instancescorrect = 0
for i in time_to_maneuver:
    if i == 5:
        instancescorrect += 1

per5 = instancescorrect/lt2m
avgtime = 5*instancescorrect
print('Percentage correct 5 s in advance: ' + str(per5))

numleft = lt2m - instancescorrect
instancescorrect = 0
for i in time_to_maneuver:
    if i == 4:
        instancescorrect += 1

per4 = instancescorrect/numleft
avgtime = avgtime + 4*instancescorrect
print('Percentage correct 4 s in advance: ' + str(per4))

numleft = numleft - instancescorrect
instancescorrect = 0
for i in time_to_maneuver:
    if i == 3:
        instancescorrect += 1

per3 = instancescorrect/numleft
avgtime = avgtime + 3*instancescorrect
print('Percentage correct 3 s in advance: ' + str(per3))

numleft = numleft - instancescorrect
instancescorrect = 0
for i in time_to_maneuver:
    if i == 2:
        instancescorrect += 1

per2 = instancescorrect/numleft
avgtime = avgtime + 2*instancescorrect
print('Percentage correct 2 s in advance: ' + str(per2))

numleft = numleft - instancescorrect
instancescorrect = 0
for i in time_to_maneuver:
    if i == 1:
        instancescorrect += 1

per1 = instancescorrect/numleft
avgtime = avgtime + 1*instancescorrect
print('Percentage correct 1 s in advance: ' + str(per1))

avgtime = avgtime/(lt2m-numleft)
print('Avg time for correct prediction: ' + str(avgtime))

Total maneuvers: 58
Percentage correct 5 s in advance: 0.5689655172413793
Percentage correct 4 s in advance: 0.24
Percentage correct 3 s in advance: 0.21052631578947367
Percentage correct 2 s in advance: 0.4666666666666667
Percentage correct 1 s in advance: 0.25
Avg time for correct prediction: 4.34


In [114]:
fillerarray = np.zeros((1,len(time_to_maneuver)))
fillerarray[0,:] = time_to_maneuver
tim2mset = np.concatenate((tim2mset,fillerarray),axis=0)
fillerarray = np.zeros((1,5,len(time_to_maneuver)))
fillerarray[0,:,:] = correct_expansion
expandedset = np.concatenate((expandedset,fillerarray),axis=0)

In [115]:
np.shape(tim2mset)

(10, 58)

In [116]:
np.shape(expandedset)

(10, 5, 58)

In [118]:
np.save("C:/Users/ykung/Downloads/b4c/timevaryingtim2mset", tim2mset)

In [119]:
np.save("C:/Users/ykung/Downloads/b4c/expandedset", expandedset)

In [121]:
num5s = 0
num4s = 0
num3s = 0
num2s = 0
num1s = 0
total = 580
tolcorr = 0
for i in range(10):
    for j in range(58):
        if tim2mset[i,j] == 5:
            num5s += 1
            tolcorr += 1
        elif tim2mset[i,j] == 4:
            num4s += 1
            tolcorr += 1
        elif tim2mset[i,j] == 3:
            num3s += 1
            tolcorr += 1
        elif tim2mset[i,j] == 2:
            num2s += 1
            tolcorr += 1
        elif tim2mset[i,j] == 1:
            num1s += 1
            tolcorr += 1
print('5s pecentage: ' + str(num5s/total))
print('4s pecentage: ' + str(num4s/(total-num5s)))
print('3s pecentage: ' + str(num3s/(total-num5s-num4s)))
print('2s pecentage: ' + str(num2s/(total-num5s-num4s-num3s)))
print('1s pecentage: ' + str(num1s/(total-num5s-num4s-num3s-num2s)))
print('total avg pred time: ' + str((num5s*5+num4s*4+num3s*3+num2s*2+num1s)/tolcorr))

5s pecentage: 0.6310344827586207
4s pecentage: 0.2616822429906542
3s pecentage: 0.3037974683544304
2s pecentage: 0.34545454545454546
1s pecentage: 0.25
total avg pred time: 4.357414448669202


In [6]:
expandedset = np.load("C:/Users/ykung/Downloads/b4c/expandedset.npy")

In [7]:
np.shape(expandedset)

(10, 5, 58)

In [8]:
num5s = 0
num4s = 0
num3s = 0
num2s = 0
num1s = 0
total = 580
for i in range(10):
    for j in range(58):
        if expandedset[i,0,j] == 1:
            num5s += 1
            
        if expandedset[i,1,j] == 1:
            num4s += 1
            
        if expandedset[i,2,j] == 1:
            num3s += 1
            
        if expandedset[i,3,j] == 1:
            num2s += 1
            
        if expandedset[i,4,j] == 1:
            num1s += 1
            
print('5s pecentage: ' + str(num5s/total))
print('4s pecentage: ' + str(num4s/total))
print('3s pecentage: ' + str(num3s/total))
print('2s pecentage: ' + str(num2s/total))
print('1s pecentage: ' + str(num1s/total))

5s pecentage: 0.6310344827586207
4s pecentage: 0.6793103448275862
3s pecentage: 0.746551724137931
2s pecentage: 0.8086206896551724
1s pecentage: 0.8241379310344827
